In [2]:
"""
This file defines the findTeachingSet function.

Utilizes the linear programming algorithm (MOSEK interior point method) in scipy.
Interior point method is O(n^3.5 * L^2 * logL * loglogL), which is polynomial.
Note, that L is the length of input in bits, so as n grows, the L terms
will act as logn. The run time is still polynomial, but log-log plot between n and runtime
would not be linear. 
"""

import numpy as np
from scipy.optimize import linprog
import warnings
warnings.filterwarnings("ignore", message="Ill-conditioned matrix ")
warnings.filterwarnings("ignore", message="Solving system with option 'cholesky':True failed. ")
warnings.filterwarnings("ignore", message="Solving system with option 'sym_pos':True failed. ")

In [13]:
def findTeachingSet(coordinates, patterns, n, d, homogeneous=False, debug=False):
    """
    In dual space, use linear programming to test if a pattern should be included in the set.
    If linear programming has a feasible solution, then include the point in the teaching set.
    If linear programming has no feasible solution, then discard the point.
    Returns optimal teaching set.
    """    
    teaching_set = [[],[]]
    if debug: debug_info = [[],[]]
    
    if not homogeneous:
        coordinates_signed = list(map(lambda x: np.append(x,[-1]), coordinates))
    else:
        coordinates_signed = coordinates
    coordinates_signed = list(map(lambda x,y:x*y, np.array(coordinates_signed), -np.array(patterns)))
    
    
    # linear progamming, but sets parameters such that we essentially only deal with a constrained linear system
    def isOptimal(i):
        
        c = coordinates_signed[i]
        A = np.array(coordinates_signed[:i] + coordinates_signed[i+1:])
        
        b = [0] * (n-1)
        
        # we include epsilon, a wiggle room, since we want to exclude the points that are "weakly optimal"
        # meaning in dual space this kind of hyperplane do not form a face of the d-cell
        # but instead, they intersect the d-cell only on a vertex (in R2) or edge (in R3), etc
        epsilon = 100 #min(abs(c))/100 if min(abs(c)) > 0 else 0.001
        #epsilon = np.finfo(np.float32).eps * 100
        
        try:
            res1 = linprog(np.zeros(len(c)), A_ub=A, b_ub=b, A_eq=c.reshape(1,len(c)), b_eq=[0], bounds=(None, None),
                     options={'cholesky':True, 'disp':False})
            #res2 = linprog(np.zeros(len(c)), A_ub=A, b_ub=b, A_eq=c.reshape(1,len(c)), b_eq=[-epsilon], bounds=(None, None),options={'cholesky':True, 'disp':False})
        except:
            return False
            
        if debug: 
            debug_info[0].append(res1['message'])
            debug_info[1].append(res2['message'])
        #if (res1['status']==0) and (np.isclose(res1['x'],0).sum() != d): print(res1['x'])
        return (res1['status']==0) and (np.isclose(res1['x'],0).sum() != d)
    
    
    for i in range(n):
        optimal = isOptimal(i)
        if optimal:
            teaching_set[0].append(coordinates[i])
            teaching_set[1].append(patterns[i])
    #print(teaching_set)
    if debug:
        for i in range(n):
            print(coordinates[i], debug_info[0][i], debug_info[1][i])
            
    return teaching_set